In [1]:
import requests
import time
import pandas as pd

# PARAMETERS
keyword = "election"
start_date = "2024-01-01"
end_date = "2024-01-03"

# Convert to Unix timestamp
start_ts = int(time.mktime(time.strptime(start_date, "%Y-%m-%d")))
end_ts = int(time.mktime(time.strptime(end_date, "%Y-%m-%d")))

In [4]:
# API endpoint
url = "https://api.pullpush.io/reddit/search/submission/"

# Empty list to store results
posts = []

# Loop through each day
current = start_ts
while current < end_ts:
    next_day = current + 86400

    params = {
        "q": keyword,
        "after": current,
        "before": next_day,
        "sort": "asc",
        "sort_type": "created_utc",
        "size": 100
    }

    response = requests.get(url, params=params)
    if response.status_code != 200:
        print(f"Error: {response.status_code}")
        current = next_day
        continue

    data = response.json().get("data", [])
    for item in data:
        posts.append({
            "title": item.get("title"),
            "content": item.get("selftext"),
            "author": item.get("author"),
            "created_utc": time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(item.get("created_utc", 0))),
            "url": "https://www.reddit.com" + item.get("permalink", "")
        })

    current = next_day
    time.sleep(1)

# Convert to DataFrame
df = pd.DataFrame(posts)
df.to_csv("reddit_results.csv", index=False, encoding="utf-8-sig")
print(df.head())

Empty DataFrame
Columns: []
Index: []
